In [ ]:
## Zeta-to-Surface Potential Conversion ##
 
## Solver of surface potential based on experimental values of zeta potential and electrolyte layer thickness (version 2.0) ##

## D.Sapalidis, Zurich, 16.12.2025 ##


import numpy as np
import pandas as pd
from scipy.optimize import brentq, newton
from pathlib import Path

def solve_surface_potential(kB, e_charge, eps0, epsr, T, zeta_mV, kappa, Rg, delta_nm):
    '''
    Surface potential solver based on: Ma, X., Li, M. & Sun, C. Effect of ionic environment in aqueous solution on nucleation 
    and stabilization of bulk nanobubbles. Appl. Surf. Sci. 656, 159726 (2024)
    '''

    vals = [kB, e_charge, eps0, epsr, T, zeta_mV, kappa, Rg, delta_nm]
    if any(pd.isna(v) for v in vals):
        return np.nan, np.nan
    if min(kB, e_charge, eps0, epsr, T, kappa, Rg) <= 0 or delta_nm < 0:
        return np.nan, np.nan

    zeta_V = zeta_mV * 1e-3
    delta_m = delta_nm * 1e-9
    V_T = (kB * T) / e_charge

    prefactor_sigma = 2.0 * eps0 * epsr * V_T * kappa
    stern_factor = delta_m / (eps0 * epsr)
    inv_kappa_Rg = 1.0 / (kappa * Rg)


   
    def f(psi):
        lhs = 2.0 * V_T * psi

        
        if abs(psi) < 1e-10:
            sinh_val = psi
            term_log_part = 1.0
            cosh_half = 1.0
        else:
            sinh_val = np.sinh(psi)
            sinh_sq = sinh_val * sinh_val
            cosh_half = np.cosh(psi / 2.0)
            term_log_part = (8.0 * np.log(cosh_half)) / sinh_sq

        sech2_half = 1.0 / (cosh_half * cosh_half)
        term_inside_sqrt = 1.0 + inv_kappa_Rg * (2.0 * sech2_half) + (inv_kappa_Rg**2) * term_log_part
        correction_factor = np.sqrt(term_inside_sqrt)

        term2 = prefactor_sigma * sinh_val * correction_factor * stern_factor
        rhs = zeta_V + term2
        return lhs - rhs

    # Initial guess
    psi_guess = zeta_V / (2.0 * V_T)

    
    grid = np.concatenate([np.linspace(-20, -1e-6, 4000), np.linspace(1e-6, 20, 4000)])
    fg = np.array([f(x) for x in grid])
    idx = np.where(np.sign(fg[:-1]) * np.sign(fg[1:]) < 0)[0]

    try:
        if len(idx) > 0:
            mids = (grid[idx] + grid[idx + 1]) / 2.0
            j = int(np.argmin(np.abs(mids - psi_guess)))
            a, b = grid[idx[j]], grid[idx[j] + 1]
            psi_sol = brentq(f, a, b, maxiter=500)
        else:
            psi_sol = newton(f, x0=psi_guess, maxiter=200)
    except Exception:
        return np.nan, np.nan

    psi0_V = 2.0 * V_T * psi_sol
    return float(psi_sol), float(psi0_V)

# Reads the excel copied input file
input_file = Path(r"C:\Users\dsapa\Desktop\extra\Non-linear Poisson-Boltzmann Calculations\Zeta-to-Surface_Input.txt")  
data = pd.read_csv(input_file, sep="\t")
data.columns = data.columns.str.strip()


numeric_cols = ['w/v(mg/ml)', 'kB (J/K)', 'e (C)', 'ε0 (F/m)', 'Τ (K)', 'εr', 'ζ (mV)', 'κ (1/m)', 'Rg (m)', 'δ (nm)']
for c in numeric_cols:
    if c in data.columns:
        data[c] = pd.to_numeric(data[c], errors="coerce")

psis = data.apply(lambda r: solve_surface_potential(
    r['kB (J/K)'], r['e (C)'], r['ε0 (F/m)'], r['εr'], r['Τ (K)'],
    r['ζ (mV)'], r['κ (1/m)'], r['Rg (m)'], r['δ (nm)']
), axis=1, result_type="expand")
psis.columns = ['Psi_agg_dimless', 'psi_agg0_V']
data = pd.concat([data, psis], axis=1)
data['psi_agg0_mV'] = 1e3 * data['psi_agg0_V']

out = pd.DataFrame({
    "Sample": data["Buffer"].astype(str) + "_" + data["son"].astype(str),
    "Dilution_wv_mgml": data["w/v(mg/ml)"],
    "Solvent": data["Buffer"],
    "Zeta_mV": data["ζ (mV)"],
    "Psi_agg_dimless": data["Psi_agg_dimless"],
    "psi_agg0_mV": data["psi_agg0_mV"],   # ψ_agg(0) in mV
})

print(out)


out.to_csv(input_file.with_name("Surface_Potential_Table.csv"), index=False)


    Sample  Dilution_wv_mgml Solvent  Zeta_mV  Psi_agg_dimless  psi_agg0_mV
0  H2O_yes            0.0625     H2O   -30.45        -0.678553   -34.252965
1  H2O_yes            0.1250     H2O   -38.37        -0.869562   -43.894977
2  H2O_yes            0.2500     H2O   -41.31        -0.937515   -47.325209
3  H2O_yes            0.5000     H2O   -44.68        -1.023075   -51.644219
4  H2O_yes            1.0000     H2O   -52.37        -1.228870   -62.032643
5  H2O_yes            2.0000     H2O   -54.34        -1.277363   -64.480541
